In [1]:
with open("Day24.txt") as file:
    swires, sgates = file.read().split("\n\n")    

In [2]:
from dataclasses import dataclass

@dataclass
class Gate:
    in1: str
    in2: str
    out: str
    op: str

    def __post_init__(self):
        # Commutative boolean law
        self.in1, self.in2 = sorted((self.in1, self.in2))
    
    def run(self, wires):
        if (out := wires.get(self.out)) is not None:
            return out
        in1, in2 = wires.get(self.in1), wires.get(self.in2)
        if in1 is None or in2 is None:
            return
        match self.op:
            case "AND":
                out = in1 & in2
            case "OR":
                out = in1 | in2
            case "XOR":
                out = in1 ^ in2
        wires[self.out] = out
        return out

In [3]:
import re

wires = {}
for line in swires.splitlines():
    name, value = line.split(": ")
    wires[name] = int(value)

gates = {}
regex = re.compile(r"(?P<in1>\w+) (?P<op>\w+) (?P<in2>\w+) -> (?P<out>\w+)")
for line in sgates.splitlines():
    if match := regex.match(line):
        gates[match.group("out")] = Gate(**match.groupdict())

In [4]:
from queue import deque

def run(wires, gates):
    wires = wires.copy()
    queue = deque(gates.values())
    while queue:
        gate = queue.popleft()
        if gate.run(wires) is None:
            queue.append(gate)
    result = 0
    for wire, value in sorted(wires.items(), reverse=True):
        if not wire.startswith("z"):
            continue
        result = (result << 1) | value
    return result

In [5]:
%%time
run(wires, gates)

CPU times: user 420 μs, sys: 0 ns, total: 420 μs
Wall time: 424 μs


53755311654662

In [6]:
def solve(gates):
    defects = set()
    for gate in gates.values():
        if gate.op != "XOR" and gate.out[0] == "z" and gate.out != "z45":
            # Every gate for Z output should be XOR except for the last one
            defects.add(gate.out)
        if gate.op == "XOR":
            if all(val[0] not in "xyz" for val in (gate.in1, gate.in2, gate.out)):
                # Every XOR gate should deal at least with x, y or z inputs/outputs
                defects.add(gate.out)
            for prev in gates.values():
                if prev.op == "OR" and gate.out in (prev.in1, prev.in2):
                    # Every XOR gate input should come from at least one OR gate
                    defects.add(gate.out)
        if gate.op == "AND" and gate.in1 != "x00":  # Start gate is an exception
            for prev in gates.values():
                if prev.op != "OR" and gate.out in (prev.in1, prev.in2):
                    # Every AND gate input should not come from any OR gate
                    defects.add(gate.out)
    return ",".join(sorted(defects))

In [7]:
%%time
solve(gates)

CPU times: user 1.66 ms, sys: 0 ns, total: 1.66 ms
Wall time: 1.66 ms


'dkr,ggk,hhh,htp,rhv,z05,z15,z20'